In [1]:
import pandas as pd
import numpy as np 
import xgboost as xgb
import patsy

In [2]:
def Gini(y_pred, dtrain):
    y_true = dtrain.get_label()
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort rows on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(0, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    return 'Gini', -G_pred/G_true

## Train Groupped

In [51]:
train = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
test = pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
full = pd.concat(objs = [train,test])
num_mask = np.array([True if obj != 'object' else False for obj in full.dtypes])
full_num = full.iloc[:, num_mask]
full_cat = full.iloc[:, ~num_mask]
full_cat = full_cat.apply(func = lambda x: np.unique(x, return_inverse =1)[1], axis=0)
full = pd.concat([full_num,full_cat], axis=1)
split = np.isnan(full.Hazard)
labels = full.loc[~split,'Hazard'].values
train = full[~split].drop('Hazard', axis=1).values
test  = full[split].drop('Hazard', axis=1).values
params = {  "objective": "reg:linear"
          , "eta": 0.005
          , "min_child_weight": 6
          , "subsample": 0.6
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

offset = 4000
num_rounds = 10000

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(params, xgtrain, num_rounds, watchlist, feval=Gini, early_stopping_rounds=120)

Will train until val error hasn't decreased in 120 rounds.
[0]	train-Gini:-0.346824	val-Gini:-0.246768
[1]	train-Gini:-0.390443	val-Gini:-0.255613
[2]	train-Gini:-0.413022	val-Gini:-0.271833
[3]	train-Gini:-0.430155	val-Gini:-0.281773
[4]	train-Gini:-0.441845	val-Gini:-0.289159
[5]	train-Gini:-0.448581	val-Gini:-0.297303
[6]	train-Gini:-0.452878	val-Gini:-0.303215
[7]	train-Gini:-0.455464	val-Gini:-0.306223
[8]	train-Gini:-0.457328	val-Gini:-0.311998
[9]	train-Gini:-0.460885	val-Gini:-0.312252
[10]	train-Gini:-0.462750	val-Gini:-0.312528
[11]	train-Gini:-0.464781	val-Gini:-0.315125
[12]	train-Gini:-0.466005	val-Gini:-0.316062
[13]	train-Gini:-0.471742	val-Gini:-0.319354
[14]	train-Gini:-0.470324	val-Gini:-0.318840
[15]	train-Gini:-0.470341	val-Gini:-0.320012
[16]	train-Gini:-0.470851	val-Gini:-0.319668
[17]	train-Gini:-0.471916	val-Gini:-0.322079
[18]	train-Gini:-0.472012	val-Gini:-0.321904
[19]	train-Gini:-0.473087	val-Gini:-0.321015
[20]	train-Gini:-0.472218	val-Gini:-0.321429
[21]	t

In [52]:
train = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
test = pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
full = pd.concat(objs = [train,test])
full.drop(['T1_V10','T2_V7', 'T2_V10'], axis=1, inplace = 1)
num_mask = np.array([True if obj != 'object' else False for obj in full.dtypes])
full_num = full.iloc[:, num_mask]
full_cat = full.iloc[:, ~num_mask]
full_cat = full_cat.apply(func = lambda x: np.unique(x, return_inverse =1)[1], axis=0)
full = pd.concat([full_num,full_cat], axis=1)
split = np.isnan(full.Hazard)
labels = full.loc[~split,'Hazard'].values
train = full[~split].drop('Hazard', axis=1).values
test  = full[split].drop('Hazard', axis=1).values
params = {  "objective": "reg:linear"
          , "eta": 0.005
          , "min_child_weight": 6
          , "subsample": 0.6
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

offset = 4000
num_rounds = 10000

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(params, xgtrain, num_rounds, watchlist, feval=Gini, early_stopping_rounds=120)

Will train until val error hasn't decreased in 120 rounds.
[0]	train-Gini:-0.347586	val-Gini:-0.238417
[1]	train-Gini:-0.395645	val-Gini:-0.262061
[2]	train-Gini:-0.420575	val-Gini:-0.283996
[3]	train-Gini:-0.434846	val-Gini:-0.297747
[4]	train-Gini:-0.442438	val-Gini:-0.303378
[5]	train-Gini:-0.448493	val-Gini:-0.306409
[6]	train-Gini:-0.452843	val-Gini:-0.312318
[7]	train-Gini:-0.456399	val-Gini:-0.317243
[8]	train-Gini:-0.457253	val-Gini:-0.315214
[9]	train-Gini:-0.457792	val-Gini:-0.317962
[10]	train-Gini:-0.458292	val-Gini:-0.321958
[11]	train-Gini:-0.461800	val-Gini:-0.325037
[12]	train-Gini:-0.464073	val-Gini:-0.324359
[13]	train-Gini:-0.464750	val-Gini:-0.325578
[14]	train-Gini:-0.465217	val-Gini:-0.327966
[15]	train-Gini:-0.466711	val-Gini:-0.327506
[16]	train-Gini:-0.467303	val-Gini:-0.326844
[17]	train-Gini:-0.467335	val-Gini:-0.327443
[18]	train-Gini:-0.468297	val-Gini:-0.328058
[19]	train-Gini:-0.468471	val-Gini:-0.327302
[20]	train-Gini:-0.468509	val-Gini:-0.328291
[21]	t

In [57]:
train = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
test = pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
full = pd.concat(objs = [train,test])
full.drop(['T1_V10','T2_V7', 'T2_V10'], axis=1, inplace = 1)
num_mask = np.array([True if obj != 'object' else False for obj in full.dtypes])
full_num = full.iloc[:, num_mask]
full_cat = full.iloc[:, ~num_mask]
full_cat = full_cat.apply(func = lambda x: np.unique(x, return_inverse =1)[1], axis=0)
full = pd.concat([full_num,full_cat], axis=1)
split = np.isnan(full.Hazard)
labels = full.loc[~split,'Hazard'].values
train = full[~split].drop('Hazard', axis=1).values
test  = full[split].drop('Hazard', axis=1).values
params = {  "objective": "reg:linear"
          , "eta": 0.003
          , "min_child_weight": 6
          , "subsample": 0.6
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

offset = 4000
num_rounds = 10000

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(params, xgtrain, num_rounds, watchlist, feval=Gini, early_stopping_rounds=120)

Will train until val error hasn't decreased in 120 rounds.
[0]	train-Gini:-0.347586	val-Gini:-0.238417
[1]	train-Gini:-0.394631	val-Gini:-0.286498
[2]	train-Gini:-0.419227	val-Gini:-0.289174
[3]	train-Gini:-0.432459	val-Gini:-0.294700
[4]	train-Gini:-0.436062	val-Gini:-0.297450
[5]	train-Gini:-0.446039	val-Gini:-0.302943
[6]	train-Gini:-0.449489	val-Gini:-0.305130
[7]	train-Gini:-0.452298	val-Gini:-0.308464
[8]	train-Gini:-0.454951	val-Gini:-0.314043
[9]	train-Gini:-0.461745	val-Gini:-0.321535
[10]	train-Gini:-0.461968	val-Gini:-0.321636
[11]	train-Gini:-0.462191	val-Gini:-0.319089
[12]	train-Gini:-0.463037	val-Gini:-0.321029
[13]	train-Gini:-0.463959	val-Gini:-0.322871
[14]	train-Gini:-0.466470	val-Gini:-0.325368
[15]	train-Gini:-0.465042	val-Gini:-0.324446
[16]	train-Gini:-0.466046	val-Gini:-0.327592
[17]	train-Gini:-0.466915	val-Gini:-0.327302
[18]	train-Gini:-0.467373	val-Gini:-0.327820
[19]	train-Gini:-0.467591	val-Gini:-0.328921
[20]	train-Gini:-0.467433	val-Gini:-0.330142
[21]	t

## Train Dummy

In [50]:
train = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
test = pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
full = pd.concat(objs = [train,test])

num_mask = np.array([True if obj != 'object' else False for obj in full.dtypes])
full_num = full.iloc[:, num_mask]
full_cat = full.iloc[:, ~num_mask]
cat_names = full_cat.columns

form = ' + '.join(cat_names)
form += ' - 1'
x_dummies = patsy.dmatrix(form, full_cat, return_type='dataframe')
full_dummies = pd.concat([full_num, x_dummies], axis = 1)

split = np.isnan(full_dummies.Hazard)
labels = full_dummies.loc[~split,'Hazard'].values
train = full_dummies[~split].drop('Hazard', axis=1).values
test  = full_dummies[split].drop('Hazard', axis=1).values

params = {  "objective": "reg:linear"
          , "eta": 0.005
          , "min_child_weight": 6
          , "subsample": .33
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

offset = 4000
num_rounds = 10000

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(params, xgtrain, num_rounds, watchlist, feval= Gini, early_stopping_rounds=120)

Will train until val error hasn't decreased in 120 rounds.
[0]	train-Gini:-0.272465	val-Gini:-0.205838
[1]	train-Gini:-0.343080	val-Gini:-0.282593
[2]	train-Gini:-0.366691	val-Gini:-0.298440
[3]	train-Gini:-0.372402	val-Gini:-0.298863
[4]	train-Gini:-0.381124	val-Gini:-0.303024
[5]	train-Gini:-0.390212	val-Gini:-0.306036
[6]	train-Gini:-0.398567	val-Gini:-0.303563
[7]	train-Gini:-0.400013	val-Gini:-0.303926
[8]	train-Gini:-0.403951	val-Gini:-0.304618
[9]	train-Gini:-0.409085	val-Gini:-0.306384
[10]	train-Gini:-0.411760	val-Gini:-0.309622
[11]	train-Gini:-0.415855	val-Gini:-0.312079
[12]	train-Gini:-0.417812	val-Gini:-0.312973
[13]	train-Gini:-0.420572	val-Gini:-0.318124
[14]	train-Gini:-0.420217	val-Gini:-0.318471
[15]	train-Gini:-0.421177	val-Gini:-0.318779
[16]	train-Gini:-0.422405	val-Gini:-0.321584
[17]	train-Gini:-0.423207	val-Gini:-0.322043
[18]	train-Gini:-0.424306	val-Gini:-0.322681
[19]	train-Gini:-0.424559	val-Gini:-0.326071
[20]	train-Gini:-0.424677	val-Gini:-0.326518
[21]	t

In [53]:
train = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
test = pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
full = pd.concat(objs = [train,test])
full.drop(['T1_V10','T2_V7', 'T2_V10'], axis=1, inplace = 1)

num_mask = np.array([True if obj != 'object' else False for obj in full.dtypes])
full_num = full.iloc[:, num_mask]
full_cat = full.iloc[:, ~num_mask]
cat_names = full_cat.columns

form = ' + '.join(cat_names)
form += ' - 1'
x_dummies = patsy.dmatrix(form, full_cat, return_type='dataframe')
full_dummies = pd.concat([full_num, x_dummies], axis = 1)

split = np.isnan(full_dummies.Hazard)
labels = full_dummies.loc[~split,'Hazard'].values
train = full_dummies[~split].drop('Hazard', axis=1).values
test  = full_dummies[split].drop('Hazard', axis=1).values

params = {  "objective": "reg:linear"
          , "eta": 0.005
          , "min_child_weight": 6
          , "subsample": .33
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

offset = 4000
num_rounds = 10000

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(params, xgtrain, num_rounds, watchlist, feval= Gini, early_stopping_rounds=120)

Will train until val error hasn't decreased in 120 rounds.
[0]	train-Gini:-0.273128	val-Gini:-0.208184
[1]	train-Gini:-0.342689	val-Gini:-0.269803
[2]	train-Gini:-0.361569	val-Gini:-0.290681
[3]	train-Gini:-0.371445	val-Gini:-0.295730
[4]	train-Gini:-0.381948	val-Gini:-0.297920
[5]	train-Gini:-0.392330	val-Gini:-0.311763
[6]	train-Gini:-0.400485	val-Gini:-0.315650
[7]	train-Gini:-0.402942	val-Gini:-0.322935
[8]	train-Gini:-0.404316	val-Gini:-0.320829
[9]	train-Gini:-0.410586	val-Gini:-0.324723
[10]	train-Gini:-0.414979	val-Gini:-0.329780
[11]	train-Gini:-0.419353	val-Gini:-0.331066
[12]	train-Gini:-0.419133	val-Gini:-0.332722
[13]	train-Gini:-0.421292	val-Gini:-0.331909
[14]	train-Gini:-0.422051	val-Gini:-0.329934
[15]	train-Gini:-0.422147	val-Gini:-0.327038
[16]	train-Gini:-0.422939	val-Gini:-0.325439
[17]	train-Gini:-0.425149	val-Gini:-0.325359
[18]	train-Gini:-0.424598	val-Gini:-0.326851
[19]	train-Gini:-0.425137	val-Gini:-0.326697
[20]	train-Gini:-0.425777	val-Gini:-0.326790
[21]	t

In [3]:
train = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
test = pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
full = pd.concat(objs = [train,test])
# full.drop(['T1_V10','T2_V7', 'T2_V10'], axis=1, inplace = 1)

num_mask = np.array([True if obj != 'object' else False for obj in full.dtypes])
full_num = full.iloc[:, num_mask]
full_cat = full.iloc[:, ~num_mask]
cat_names = full_cat.columns

form = ' + '.join(cat_names)
form += ' - 1'
x_dummies = patsy.dmatrix(form, full_cat, return_type='dataframe')
full_dummies = pd.concat([full_num, x_dummies], axis = 1)

split = np.isnan(full_dummies.Hazard)
labels = full_dummies.loc[~split,'Hazard'].values
train = full_dummies[~split].drop('Hazard', axis=1).values
test  = full_dummies[split].drop('Hazard', axis=1).values

params = {  "objective": "reg:linear"
          , "eta": 0.001
          , "min_child_weight": 50
          , "subsample": .33
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

offset = 10000
num_rounds = 10000

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(params, xgtrain, num_rounds, watchlist, feval= Gini, early_stopping_rounds=200)

Will train until val error hasn't decreased in 200 rounds.
[0]	train-Gini:-0.309787	val-Gini:-0.290910
[1]	train-Gini:-0.344539	val-Gini:-0.324592
[2]	train-Gini:-0.361855	val-Gini:-0.333185
[3]	train-Gini:-0.371071	val-Gini:-0.341021
[4]	train-Gini:-0.376489	val-Gini:-0.341313
[5]	train-Gini:-0.379705	val-Gini:-0.343369
[6]	train-Gini:-0.380993	val-Gini:-0.342350
[7]	train-Gini:-0.385191	val-Gini:-0.346473
[8]	train-Gini:-0.387348	val-Gini:-0.347261
[9]	train-Gini:-0.387457	val-Gini:-0.346934
[10]	train-Gini:-0.388700	val-Gini:-0.348373
[11]	train-Gini:-0.389898	val-Gini:-0.349272
[12]	train-Gini:-0.390067	val-Gini:-0.351497
[13]	train-Gini:-0.391045	val-Gini:-0.353766
[14]	train-Gini:-0.391888	val-Gini:-0.353678
[15]	train-Gini:-0.392729	val-Gini:-0.354423
[16]	train-Gini:-0.394223	val-Gini:-0.354342
[17]	train-Gini:-0.395501	val-Gini:-0.354302
[18]	train-Gini:-0.396736	val-Gini:-0.356208
[19]	train-Gini:-0.397627	val-Gini:-0.356640
[20]	train-Gini:-0.397745	val-Gini:-0.357890
[21]	t

In [15]:
train = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
test = pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
full = pd.concat(objs = [train,test])
full.drop(['T1_V10','T2_V7', 'T2_V10'], axis=1, inplace = 1)

num_mask = np.array([True if obj != 'object' else False for obj in full.dtypes])
full_num = full.iloc[:, num_mask]
full_cat = full.iloc[:, ~num_mask]
cat_names = full_cat.columns

form = ' + '.join(cat_names)
form += ' - 1'
x_dummies = patsy.dmatrix(form, full_cat, return_type='dataframe')
full_dummies = pd.concat([full_num, x_dummies], axis = 1)

split = np.isnan(full_dummies.Hazard)
labels = full_dummies.loc[~split,'Hazard'].values
train = full_dummies[~split].drop('Hazard', axis=1).values
test  = full_dummies[split].drop('Hazard', axis=1).values

params = {  "objective": "reg:linear"
          , "eta": 0.001
          , "min_child_weight": 50
          , "subsample": .33
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

offset = 10000
num_rounds = 10000

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(params, xgtrain, num_rounds, watchlist, feval= Gini, early_stopping_rounds=200)

Will train until val error hasn't decreased in 200 rounds.
[0]	train-Gini:-0.294725	val-Gini:-0.273771
[1]	train-Gini:-0.326703	val-Gini:-0.297560
[2]	train-Gini:-0.345668	val-Gini:-0.322357
[3]	train-Gini:-0.355421	val-Gini:-0.330886
[4]	train-Gini:-0.360529	val-Gini:-0.335885
[5]	train-Gini:-0.368581	val-Gini:-0.337330
[6]	train-Gini:-0.370719	val-Gini:-0.338461
[7]	train-Gini:-0.370390	val-Gini:-0.338254
[8]	train-Gini:-0.371507	val-Gini:-0.337056
[9]	train-Gini:-0.374013	val-Gini:-0.338378
[10]	train-Gini:-0.375351	val-Gini:-0.339399
[11]	train-Gini:-0.380350	val-Gini:-0.341883
[12]	train-Gini:-0.381665	val-Gini:-0.344418
[13]	train-Gini:-0.382837	val-Gini:-0.347129
[14]	train-Gini:-0.384322	val-Gini:-0.347297
[15]	train-Gini:-0.383144	val-Gini:-0.346023
[16]	train-Gini:-0.384455	val-Gini:-0.347005
[17]	train-Gini:-0.384314	val-Gini:-0.345533
[18]	train-Gini:-0.386091	val-Gini:-0.347314
[19]	train-Gini:-0.386919	val-Gini:-0.347680
[20]	train-Gini:-0.389078	val-Gini:-0.349484
[21]	t

In [60]:
train = pd.read_csv('./train.csv', sep = ',', index_col = 'Id')
test = pd.read_csv('./test.csv', sep = ',',index_col = 'Id')
full = pd.concat(objs = [train,test])
full.drop(['T1_V10','T2_V7','T2_V10'], axis=1, inplace = 1)

num_mask = np.array([True if obj != 'object' else False for obj in full.dtypes])
full_num = full.iloc[:, num_mask]
full_cat = full.iloc[:, ~num_mask]
cat_names = full_cat.columns

form = ' + '.join(cat_names)
form += ' - 1'
x_dummies = patsy.dmatrix(form, full_cat, return_type='dataframe')
full_dummies = pd.concat([full_num, x_dummies], axis = 1)

split = np.isnan(full_dummies.Hazard)
labels = full_dummies.loc[~split,'Hazard'].values
train = full_dummies[~split].drop('Hazard', axis=1).values
test  = full_dummies[split].drop('Hazard', axis=1).values

params = {  "objective": "reg:linear"
          , "eta": 0.001
          , "min_child_weight": 50
          , "subsample": .6
          , "colsample_bytree": 0.7
          , "scale_pos_weight": 1
          , "silent": 1
          , "max_depth": 9}

offset = 10000
num_rounds = 10000

xgtest = xgb.DMatrix(test)

xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])

watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(params, xgtrain, num_rounds, watchlist, feval= Gini, early_stopping_rounds=200)

Will train until val error hasn't decreased in 200 rounds.
[0]	train-Gini:-0.317131	val-Gini:-0.293640
[1]	train-Gini:-0.343886	val-Gini:-0.315892
[2]	train-Gini:-0.363800	val-Gini:-0.329998
[3]	train-Gini:-0.377254	val-Gini:-0.339919
[4]	train-Gini:-0.382027	val-Gini:-0.340071
[5]	train-Gini:-0.386133	val-Gini:-0.339138
[6]	train-Gini:-0.391470	val-Gini:-0.341351
[7]	train-Gini:-0.393012	val-Gini:-0.341360
[8]	train-Gini:-0.395975	val-Gini:-0.344601
[9]	train-Gini:-0.398260	val-Gini:-0.347532
[10]	train-Gini:-0.398153	val-Gini:-0.347840
[11]	train-Gini:-0.397826	val-Gini:-0.345964
[12]	train-Gini:-0.401141	val-Gini:-0.348585
[13]	train-Gini:-0.403948	val-Gini:-0.351051
[14]	train-Gini:-0.403791	val-Gini:-0.351685
[15]	train-Gini:-0.404382	val-Gini:-0.351319
[16]	train-Gini:-0.405335	val-Gini:-0.352920
[17]	train-Gini:-0.405687	val-Gini:-0.353733
[18]	train-Gini:-0.405926	val-Gini:-0.354113
[19]	train-Gini:-0.407217	val-Gini:-0.355228
[20]	train-Gini:-0.406657	val-Gini:-0.355302
[21]	t